<a href="https://colab.research.google.com/github/Abdulazizkoja1/T5/blob/main/RAG_Exam_(4).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Traffic Violation RAG System
In this exam, you will implement a Retrieval-Augmented Generation (RAG) system that uses a language model and a vector database to answer questions about traffic violations. The goal is to generate answers with relevant data based on a dataset of traffic violations and fines.

Here are helpful resources:
* [LangChain](https://www.langchain.com/)
* [groq cloud documentation](https://console.groq.com/docs/models)
* [LangChain HuggingFace](https://python.langchain.com/docs/integrations/text_embedding/sentence_transformers/)
* [Chroma Vector Store](https://python.langchain.com/docs/integrations/vectorstores/chroma/)
* [Chroma Website](https://docs.trychroma.com/getting-started)
* [ChatGroq LangChain](https://python.langchain.com/docs/integrations/chat/groq/)
* [LLM Chain](https://api.python.langchain.com/en/latest/chains/langchain.chains.llm.LLMChain.html#langchain.chains.llm.LLMChain)

Dataset [source](https://www.moi.gov.sa/wps/portal/Home/sectors/publicsecurity/traffic/contents/!ut/p/z0/04_Sj9CPykssy0xPLMnMz0vMAfIjo8ziDTxNTDwMTYy83V0CTQ0cA71d_T1djI0MXA30gxOL9L30o_ArApqSmVVYGOWoH5Wcn1eSWlGiH1FSlJiWlpmsagBlKCQWqRrkJmbmqRqUZebngB2gUJAKdERJZmqxfkG2ezgAhzhSyw!!/)

Some installs if needed:
```python
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain
```

In [ ]:
# !kaggle datasets download -d khaledzsa/dataset
# !unzip dataset.zip

## Step 1: Install Required Libraries

To begin, install the necessary libraries for this project. The libraries include `LangChain` for building language model chains, and `Chroma` for managing a vector database.

In [ ]:
# !pip install langchain langchain_community unstructured sentence_transformers tiktoken chromadb langchain_chroma langchain_groq

## Step 2: Load the Traffic Violations Dataset

You are provided with a dataset of traffic violations. Load the CSV file into a pandas DataFrame and preview the first few rows of the dataset using `.head()`. You can also try and see the dataset's characteristics.

In [ ]:
import os
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
import markdown
from langchain.text_splitter import RecursiveCharacterTextSplitter


df = pd.read_csv('Dataset.csv')

df.head()

,المخالفة,الغرامة
0,قيادة المركبة في الأسواق التي لا يسمح بالقيادة...,الغرامة المالية 100 - 150 ريال
1,ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.,الغرامة المالية 100 - 150 ريال
2,عدم وجود تأمين ساري للمركبة.,الغرامة المالية 100 - 150 ريال
3,عبور المشاة للطرق من غير الأماكن المخصصة لهم.,الغرامة المالية 100 - 150 ريال
4,عدم تقيد المشاة بالإشارات الخاصة بهم.,الغرامة المالية 100 - 150 ريال


## Step 3: Create Markdown Content from the Dataset

For each traffic violation in the dataset, you will generate markdown text that describes the violation and the associated fine. Create a loop to iterate through the dataset and store the generated markdown in a list. Each fine should look like this:

**المخالفة** - الغرامة

In [ ]:
directory = 'data/markdown_files'
os.makedirs(directory, exist_ok=True)

In [ ]:
for i in range(len(df)):

    title = df['المخالفة'].iloc[i]
    content = df['الغرامة'].iloc[i]

    markdown_content = f"# {title}\n\n"
    markdown_content += f"{content}\n\n"

    with open(f'{directory}/{i}.md', 'w', encoding='utf-8') as file:
        file.write(markdown_content)

In [ ]:
markdown_texts = []
for filename in os.listdir(directory):
  if filename.endswith(".md"):
    with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
      markdown_content = file.read()
      html_content = markdown.markdown(markdown_content)
      markdown_texts.append(html_content)

## Step 4: Chunk the Markdown Data

Using LangChain's `RecursiveCharacterTextSplitter`, split the markdown texts into smaller chunks that will be stored in the vector database.

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
documents = text_splitter.create_documents(markdown_texts)

## Step 5: Generate Embeddings for the Documents

Generate embeddings for the chunks of text using HuggingFace's pre-trained Arabic language model. These embeddings will be stored in a `Chroma` vector store.

In [ ]:
import os
import json
from langchain_chroma import Chroma
from langchain.chains import RetrievalQA
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

In [ ]:
PRESIST_DIRECTORY = '/content/chroma_db'
embedding_function = SentenceTransformerEmbeddings(model_name="aubmindlab/bert-base-arabertv2")
persist_directory = "./chroma_db"
db = Chroma(persist_directory=persist_directory, embedding_function=embedding_function)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## Step 6: Define the RAG Prompt Template

Define a custom prompt template in Arabic to retrieve traffic violation-related answers based on the context. Ensure the template greets the user first, states that the information provided could be incorrect, and advises the user to visit the traffic initiative website to verify. Additionally, provide the user with advice in Arabic, ensuring it stays within the given context.

In [ ]:
PROMPT_TEMPLATE="""
  انت ضابط مرور، يجب عليك ان الرجوع الى البيانات المعطاه لك لاعطاء الغرامة المناسبة. و اعطي الاجابة بالعربي
  بالاضافة ابدأ بالسلام في بداية الجملة.
  و في النهاية انصح المخالف
  و اذا ليست المخالفة موجودة فلا تجاوب و قل لا اعلم الاجابة
  المعلومات: {context}
  الغرامة: {question}
  إجابتك:
"""

prompt_template = PromptTemplate(
    template=PROMPT_TEMPLATE, input_variables=["context", "question"]
)

## Step 7: Initialize the Language Model

Initialize the language model using the Groq API. Set up the model with a specific configuration, including the API key, temperature setting, and model name.

In [ ]:
groq_api_key = ""
llm = ChatGroq(temperature=0, groq_api_key=groq_api_key, model_name="llama3-8b-8192")

## Step 8: Create the LLM Chain

Now, you will create an LLM Chain that combines the language model and the prompt template you defined. This chain will be used to generate responses based on the retrieved context.

In [ ]:
MODEL = LLMChain(llm=llm,
                 prompt=prompt_template,
                 verbose=True)

## Step 9: Implement the Query Function

Create a function `query_rag` that will take a user query as input, retrieve relevant context from the vector store, and use the language model to generate a response based on that context.

In [ ]:
def query_rag(query: str):
    similarity_search_results = db.similarity_search_with_score(query, k=4)
    context_text = "\n\n".join([doc.page_content for doc, _score in similarity_search_results])

    rag_response = MODEL.invoke({"context": context_text, "question": query})

    return rag_response

## Step 10: Inference - Running Queries in the RAG System

In this final step, you will implement an inference pipeline to handle real-time queries. You will allow the system to retrieve the most relevant violations and fines based on a user's input and generate a response.

1. Inference Workflow:

  * The user inputs a query (e.g., "ماهي عقوبة عدم الوقوف وقوفاً تاماً عند إشارة؟").
  * The system searches for the most relevant context from the traffic violation vector store.
  * It generates an answer and advice based on the context.

2. Goal:
  * Run the inference to answer questions based on the traffic violation dataset.

In [ ]:
response = query_rag("ماهي عقوبة عدم الوقوف وقوفاً تاماً عند إشارة؟")
response

Prompt after formatting:

  انت ضابط مرور، يجب عليك ان الرجوع الى البيانات المعطاه لك لاعطاء الغرامة المناسبة. و اعطي الاجابة بالعربي
  بالاضافة ابدأ بالسلام في بداية الجملة.
  و في النهاية انصح المخالف
  المعلومات: 
  الغرامة: ماهي عقوبة عدم الوقوف وقوفاً تاماً عند إشارة؟
  إجابتك:


> Finished chain.


{'context': '',
 'question': 'ماهي عقوبة عدم الوقوف وقوفاً تاماً عند إشارة؟',
 'text': 'سلام عليكم،\n\nوفقاً لبياناتي، فإن الغرامة المناسبة لعدم الوقوف وقوفاً تاماً عند إشارة هي 250 ريال.\n\nأنا أنصح المخالف بالتزام الإشارات المرورية والوقوف وقوفاً تاماً عند الإشارات لمنع حدوث أي مشاكل أو حوادث على الطريق.\n\nأمل أن يتعلم المخالف من هذه التجربة ويحافظ على السلامة على الطريق في المستقبل.\n\nأشكرك على الاهتمام بالسلامة على الطريق.'}

In [ ]:
response['text']

'سلام عليكم،\n\nوفقاً لبياناتي، فإن الغرامة المناسبة لعدم الوقوف وقوفاً تاماً عند إشارة هي 250 ريال.\n\nأنا أنصح المخالف بالتزام الإشارات المرورية والوقوف وقوفاً تاماً عند الإشارات لمنع حدوث أي مشاكل أو حوادث على الطريق.\n\nأمل أن يتعلم المخالف من هذه التجربة ويحافظ على السلامة على الطريق في المستقبل.\n\nأشكرك على الاهتمام بالسلامة على الطريق.'

In [ ]:
import discord
import nest_asyncio
import asyncio

nest_asyncio.apply()

intents = discord.Intents.default()
intents.message_content = True

client = discord.Client(intents=intents)

@client.event
async def on_ready():
    print(f'We have logged in as {client.user}')

@client.event
async def on_message(message):
    if message.author == client.user:
        return

    if message.content.startswith('!ask'):
      question = message.content[4:]
      response = query_rag(question)
      await message.channel.send(response['text'])


    if message.content.startswith('!hello'):
      await message.channel.send('Hello! This bot is running on Google Colab!')



client.run(token)

import time
while True:
    time.sleep(10)


2024-09-19 16:06:05 INFO     discord.client logging in using static token
2024-09-19 16:06:05 INFO     discord.client logging in using static token
INFO:discord.client:logging in using static token
2024-09-19 16:06:06 INFO     discord.gateway Shard ID None has connected to Gateway (Session ID: 9b16605f0e86ffa4904c8471123f8036).
2024-09-19 16:06:06 INFO     discord.gateway Shard ID None has connected to Gateway (Session ID: 9b16605f0e86ffa4904c8471123f8036).
INFO:discord.gateway:Shard ID None has connected to Gateway (Session ID: 9b16605f0e86ffa4904c8471123f8036).


We have logged in as T5-Answering-Bot#7079


Prompt after formatting:

  انت ضابط مرور، يجب عليك ان الرجوع الى البيانات المعطاه لك لاعطاء الغرامة المناسبة. و اعطي الاجابة بالعربي
  بالاضافة ابدأ بالسلام في بداية الجملة.
  و في النهاية انصح المخالف
  المعلومات: 
  الغرامة:  عدم وجود تأمين ساري للمركبة.
  إجابتك:


> Finished chain.


Prompt after formatting:

  انت ضابط مرور، يجب عليك ان الرجوع الى البيانات المعطاه لك لاعطاء الغرامة المناسبة. و اعطي الاجابة بالعربي
  بالاضافة ابدأ بالسلام في بداية الجملة.
  و في النهاية انصح المخالف
  المعلومات: 
  الغرامة:  ماهي عقوبة عدم الوقوف وقوفاً تاماً عند إشارة؟
  إجابتك:


> Finished chain.


Prompt after formatting:

  انت ضابط مرور، يجب عليك ان الرجوع الى البيانات المعطاه لك لاعطاء الغرامة المناسبة. و اعطي الاجابة بالعربي
  بالاضافة ابدأ بالسلام في بداية الجملة.
  و في النهاية انصح المخالف
  المعلومات: 
  الغرامة:  ماهو التفاح
  إجابتك:


> Finished chain.


KeyboardInterrupt: 

In [ ]:
# !pip install discord